In [28]:
from pathlib import Path
from datetime import datetime

def is_bank_business_day(
    date: str | datetime,
    calendar_file_path: str | Path,
) -> bool | None:
    """
    銀行営業日判定関数
    
    Args:
        date (str | datetime): 判定対象の日付（文字列またはdatetime型）
        calendar_file_path (str): 銀行カレンダーファイルのパス
        
    Returns:
        bool: 銀行営業日の場合はTrue、銀行休業日の場合はFalse
        None: 判定不能の場合
        
    Raises:
        FileNotFoundError: 銀行カレンダーファイルが存在しない場合
        ValueError: 日付のフォーマットが正しくない場合
        TypeError: 引数の型が正しくない場合
    """
    # 銀行カレンダーファイルを読み込む
    calendar_file_path = Path(calendar_file_path)
    if not calendar_file_path.exists():
        raise FileNotFoundError(f"銀行カレンダーファイルが見つかりません: {calendar_file_path}")

    # 入れ物
    closed_days = set()
    operation_days = set()

    # カレンダーファイル処理
    with calendar_file_path.open('r') as file:
        for line in file:
            striped_line = line.strip()
            if striped_line:
                key, value = striped_line.split('=', maxsplit=1)
                if key == 'cl':
                    closed_days.add(value)
                elif key == 'op':
                    operation_days.add(value)
                else:
                    raise ValueError(f"無効なフォーマット: {line}")

    # 日付の方をチェックし必要であれば変換する
    if isinstance(date, datetime):
        date_string = date.strftime('%Y/%m/%d')
    elif isinstance(date, str):
        try:
            # 日付に変換できる文字列を判定
            datetime.strptime(date, '%Y/%m/%d')
            date_string = date
        except ValueError:
            raise ValueError(f'日付のフォーマットが正しくありません: {date}')
    else:
        raise TypeError(f"日付は文字列もしくはdatetime型でなければなりません: {type(date)}")

    # 判定対象の日付が休業日リストに含まれている場合
    if date_string in closed_days:
        return False

    if date_string in operation_days:
        return True

    # 判定不能の場合(カレンダーファイルに含まれていない日付を指定した場合を想定 )
    return None

In [29]:
cal_file_path = '/developer/library_dev/project_1/src/share/receive/bank_cal.txt'
print(is_bank_business_day('2024/01/30', cal_file_path))
print(is_bank_business_day('2024/01/31', cal_file_path))
print(is_bank_business_day('2024/02/03', cal_file_path))

True
True
False


In [31]:
print(is_bank_business_day('2050/01/30', cal_file_path))

None


In [32]:
cal_file_path = '/developer/library_dev/project_1/src/share/receive/bank_cal.txta'
print(is_bank_business_day('2024/01/30', cal_file_path))

FileNotFoundError: 銀行カレンダーファイルが見つかりません: /developer/library_dev/project_1/src/share/receive/bank_cal.txta

In [33]:
def is_bank_business_day(date: str | datetime, calendar_file_path: str | Path) -> bool | None:
    """
    銀行営業日判定関数
    JP1提供銀行カレンダーに基づき、指定日付が銀行営業日であるかの判定を行う

    Args:
        date (str | datetime): 判定対象の日付(文字列またはdatetime型)
        calendar_file_path (str): 銀行カレンダーファイルのパス

    Returns:
        bool: 銀行営業日の場合はTrue、銀行休業日の場合はFalse
        None: 判定不能の場合

    Raises:
        FileNotFoundError: 銀行カレンダーファイルが存在しない場合
        ValueError: 日付のフォーマットが正しくない場合
        TypeError: 引数の型が正しくない場合
    """
    # 銀行カレンダーファイルを読み込む
    calendar_file_path = Path(calendar_file_path)
    try:
        if not calendar_file_path.exists():
            raise FileNotFoundError(f"銀行カレンダーファイルが見つかりません: {calendar_file_path}")
    except FileNotFoundError as e:
        log_msg(str(e))
        raise

    # 入れ物
    closed_days = set()
    operation_days = set()

    # カレンダーファイル処理
    try:
        with calendar_file_path.open('r') as file:
            for line in file:
                striped_line = line.strip()
                key, value = striped_line.split('=', maxsplit=1)
                if key == 'cl':
                    closed_days.add(value)
                elif key == 'op':
                    operation_days.add(value)
                else:
                    raise ValueError(f"無効なフォーマット: {line}")
    except ValueError as e:
        log_msg(str(e))
        raise

    # 日付の型をチェックし必要であれば変換する
    try:
        if isinstance(date, datetime):
            date_string = date.strftime('%Y/%m/%d')
        elif isinstance(date, str):
            datetime.strptime(date, '%Y/%m/%d')
            date_string = date
        else:
            raise TypeError(f"日付は文字列もしくはdatetime型でなければなりません: {type(date)}")
    except (ValueError, TypeError) as e:
        log_msg(str(e))
        raise

    # 判定対象の日付が休業日リストに含まれている場合
    if date_string in closed_days:
        return False
    if date_string in operation_days:
        return True

    # 判定不能の場合(カレンダーファイルに含まれていない日付を指定した場合を想定)
    return None

In [34]:
cal_file_path = '/developer/library_dev/project_1/src/share/receive/bank_cal.txt'
print(is_bank_business_day('2024/01/30', cal_file_path))
print(is_bank_business_day('2024/01/31', cal_file_path))
print(is_bank_business_day('2024/02/03', cal_file_path))

True
True
False


In [37]:
def load_calendar_file(calendar_file_path: str | Path) -> tuple[set[str], set[str]]:
    """
    銀行カレンダーファイルを読み込み、休業日と営業日のセットを返す

    Args:
        calendar_file_path (str | Path): 銀行カレンダーファイルのパス

    Returns:
        tuple[set[str], set[str]]: 休業日と営業日のセット

    Raises:
        FileNotFoundError: 銀行カレンダーファイルが存在しない場合
        ValueError: 無効なフォーマットの行がある場合
    """
    calendar_file_path = Path(calendar_file_path)
    if not calendar_file_path.exists():
        raise FileNotFoundError(f"銀行カレンダーファイルが見つかりません: {calendar_file_path}")

    closed_days = set()
    operation_days = set()
    with calendar_file_path.open('r') as file:
        for line in file:
            striped_line = line.strip()
            key, value = striped_line.split('=', maxsplit=1)
            if key == 'cl':
                closed_days.add(value)
            elif key == 'op':
                operation_days.add(value)
            else:
                raise ValueError(f"無効なフォーマット: {line}")
    return closed_days, operation_days


def _convert_date_to_string(date: str | datetime) -> str:
    """
    日付をYYYY/MM/DD形式の文字列に変換する

    Args:
        date (str | datetime): 変換する日付

    Returns:
        str: YYYY/MM/DD形式の日付文字列

    Raises:
        ValueError: 日付のフォーマットが正しくない場合
        TypeError: 引数の型が正しくない場合
    """
    if isinstance(date, datetime):
        return date.strftime('%Y/%m/%d')
    elif isinstance(date, str):
        datetime.strptime(date, '%Y/%m/%d')
        return date
    else:
        raise TypeError(f"日付は文字列もしくはdatetime型でなければなりません: {type(date)}")


def is_bank_business_day(date: str | datetime, calendar_file_path: str | Path) -> bool | None:
    """
    銀行営業日判定関数
    JP1提供銀行カレンダーに基づき、指定日付が銀行営業日であるかの判定を行う

    Args:
        date (str | datetime): 判定対象の日付(文字列またはdatetime型)
        calendar_file_path (str): 銀行カレンダーファイルのパス

    Returns:
        bool: 銀行営業日の場合はTrue、銀行休業日の場合はFalse
        None: 判定不能の場合

    Raises:
        FileNotFoundError: 銀行カレンダーファイルが存在しない場合
        ValueError: 日付のフォーマットが正しくない場合
        TypeError: 引数の型が正しくない場合
    """
    try:
        closed_days, operation_days = load_calendar_file(calendar_file_path)
    except (FileNotFoundError, ValueError) as e:
        log_msg(str(e))
        raise

    try:
        date_string = _convert_date_to_string(date)
    except (ValueError, TypeError) as e:
        log_msg(str(e))
        raise

    if date_string in closed_days:
        return False
    if date_string in operation_days:
        return True
    return None

In [38]:
cal_file_path = '/developer/library_dev/project_1/src/share/receive/bank_cal.txt'
print(is_bank_business_day('2024/01/30', cal_file_path))
print(is_bank_business_day('2024/01/31', cal_file_path))
print(is_bank_business_day('2024/02/03', cal_file_path))

True
True
False
